Install the packages

In [ ]:
!pip install magenta

In [ ]:
!pip install tensorflow_gan

Change the name of the samples

In [3]:
!python gansynth/training/pad.py data mysamples

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
guitar1.wav
  length: 24235 samples
  -> -1_32.wav
guitar2.wav
  length: 387755 samples
  will be truncated
  -> -2_32.wav
guitar3.wav
  length: 24235 samples
  -> -3_32.wav
guitar4.wav
  l

Create a the dataset of the samples

In [4]:
! python gansynth/training/make_dataset.py --in_dir mysamples --out_dir mydataset

scanning mysamples for wav files matching "[name]_[pitch].wav"...

validating wav files (sample rate: 16000, channels: 1, length: 64000, format: int16)...

-1
  pitches: [32]
-2
  pitches: [32]
-3
  pitches: [32]
-4
  pitches: [32]

writing metadata to mydataset/meta.json...
writing TFRecords to mydataset/data.tfrecord...


Train the model

In [5]:
! gansynth_train \
!     --config=mel_prog_hires \
!     --hparams='{"train_data_path":"mydataset/data.tfrecord", "train_meta_path":"mydataset/meta.json", "train_root_dir":"mymodel", "tfds_data_dir":"gs://tfds-data/datasets"}'

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit

Instructions for updating:
non-resource variables are not supported in the long term
2023-01-26 11:47:25.576235: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit

Postprocess of the model

In [18]:
!python gansynth/training/postprocess_model.py  --ckpt_dir mymodel --meta_path mydataset/meta.json

found 1 stage directories
latest stage with checkpoint: mymodel/stage_00000
latest iteration with data, index and meta files: 347136

the following postprocessing operations are required:
  - delete mymodel/meta.json
  - copy mydataset/meta.json to mymodel/meta.json
  - clean up mymodel/experiment.json
  - set relative model_checkpoint_path in mymodel/stage_00000/checkpoint

make sure your data is backed up and run this command with the --execute flag to perform the operations


In [19]:
!python gansynth/training/postprocess_model.py  --ckpt_dir mymodel --meta_path mydataset/meta.json --execute

found 1 stage directories
latest stage with checkpoint: mymodel/stage_00000
latest iteration with data, index and meta files: 347136

performing postprocessing operations...
  - delete mymodel/meta.json
  - copy mydataset/meta.json to mymodel/meta.json
  - clean up mymodel/experiment.json
  - set relative model_checkpoint_path in mymodel/stage_00000/checkpoint


Generate a new sound

In [20]:
! gansynth_generate --ckpt_dir=mymodel --output_dir=generated

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit

Instructions for updating:
non-resource variables are not supported in the long term
2023-01-24 19:42:16.886923: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit

Create new song with melody that you created with the musegan model and get the sound from magenta model.
- Place you song in the folder "midi_files".
- Change the midi_file name to your song name.

In [ ]:
! gansynth_generate --ckpt_dir=mymodel --output_dir=generated --midi_file=midi_files/Yoursong.mid